### Preprocessing  
Sofia, Angel

In [64]:
from zipfile import ZipFile, ZipInfo
import pandas as pd

## this is the zipped folder name on my computer, idk if its the same for everyone
zipped_name = 'activity+recognition+with+healthy+older+people+using+a+batteryless+wearable+sensor.zip'

## honestly, idk what this is doing, but i feel its important stuff
z = ZipFile(zipped_name, 'r')
ls1 = z.infolist()
ls2 = [file for file in ls1 if file.file_size > 0]


In [65]:
## im going to seperate the files into four categories to make adding extra attributes easier
S1F = []
S1M = []
S2F = []
S2M = []
## this one is just to check my work
other = [] #ignore

## i feel like there was a more efficient way to do this
## but this segment of code extracts the content of the files
 # and converts them into pandas dataframes
for i in range(len(ls2)):
    temp_zip = z.extract(member=ls2[i])
    if (("S1_Dataset" in temp_zip) and ("F" in temp_zip)):
        temp_csv = pd.read_csv(temp_zip, header=None)
        S1F.append(temp_csv)
    elif (("S1_Dataset" in temp_zip) and ('M' in temp_zip) and ('.txt' not in temp_zip)):
        temp_csv = pd.read_csv(temp_zip, header=None)
        S1M.append(temp_csv)
    elif (("S2_Dataset" in temp_zip) and ("F" in temp_zip)):
        temp_csv = pd.read_csv(temp_zip, header=None)
        S2F.append(temp_csv)
    elif (("S2_Dataset" in temp_zip) and ("M" in temp_zip) and ('.txt' not in temp_zip)):
        temp_csv = pd.read_csv(temp_zip, header=None)
        S2M.append(temp_csv)
    else:
        other.append(temp_zip)

In [66]:
## all dataframes in one list, seperated into four categories
 # (based on gender and room)
all_datasets = [S1F, S1M, S2F, S2M]

features = [
    'time',
    'frontal accel',
    'vertical accel',
    'lateral accel',
    'antenna id',
    'rssi',
    'phase',
    'frequency',
    'activity',
]

activities = [
    'sitting on bed',
    'sitting on chair',
    'lying',
    'ambulating'
]

## in this segment of code, i rename the headers
new_headers = dict(enumerate(features))
for subset in all_datasets:
    for df in subset:
        df.rename(columns = new_headers, inplace= True)


In [67]:
##  CAN ONLY RUN ONCE APPARENTLY
##

## adding F/M and room columns to dataframes F=0 and M=1 
for df in S1F:
    df.insert(len(df.columns) - 1, 'gender', 0)
    df.insert(1, 'room', 1)

for df in S1M:
    df.insert(len(df.columns) - 1, 'gender', 1)
    df.insert(1, 'room', 1)

for df in S2F:
    df.insert(len(df.columns) - 1, 'gender', 0)
    df.insert(1, 'room', 2)

for df in S2M:
    df.insert(len(df.columns) - 1, 'gender', 1)
    df.insert(1, 'room', 2)


We can go through each dataframe and add additional columns now. In these next cells, I add the 'consecutiveness' feature. This tells us how many times in a row an individual did an activity. 

For example, the first observation will have a 1 in this column. If the individual continued to do the same activity in the next observation, that obervation will have a 2 for the 'consecutivness' feature.
Otherwise, the count will start over at 1.

In [68]:
## function to add in the 'consecutiveness' attribute in a dataframe
def count_activities(dataframe):
    counts = []
    curr_count = 0
    prev_act = None

    for act in df['activity']:
        if act == prev_act:
            curr_count += 1
        else:
            curr_count = 1
            prev_act = act
    
        counts.append(curr_count)

    dataframe.insert(len(dataframe.columns) - 1, 'consecutiveness', counts)
    return dataframe

list_of_dataframes = S1F + S1M + S2F + S2M

##   ONLY RUN ONCE
##

## adds the column 'consecutiveness' into each dataframe
for df in list_of_dataframes:
    count_activities(df)


This next section of code adds the acceleration column.

In [69]:
import numpy as np 

def calc_and_add_accel(dataframe):
    acceleration = []
    for i in range(dataframe.shape[0]):
        fa = dataframe['frontal accel'][i]
        la = dataframe['lateral accel'][i]
        va = dataframe['vertical accel'][i]

        accel_vector = np.sqrt(fa**2 + la**2 + va**2)
        acceleration.append(accel_vector)

    dataframe['acceleration'] = acceleration
    return dataframe


In [70]:
## JUST RUN ONCE

for df in list_of_dataframes:
    calc_and_add_accel(df)

Added binary activity status in the last column. 0 = non-active, 1 = active


I also combined all observations into one large dataframe. 

In [71]:
large_df = pd.concat(list_of_dataframes, ignore_index=True)

In [72]:
totals = large_df.shape[0]

bed = (large_df['activity'] == 1).sum() / totals
chair = (large_df['activity'] == 2).sum() / totals
lying = (large_df['activity'] == 3).sum() / totals 
amb = (large_df['activity'] == 4).sum() / totals 

act_dict = {1: bed, 2: chair, 3: lying, 4: amb}

large_df['freq'] = large_df['activity'].map(act_dict)

In [73]:
binary_activity = []
for i in range(large_df.shape[0]):
    if large_df['activity'][i] in range(0,4):
        binary_activity.append(0)
    else:
        binary_activity.append(1)

## ONLY RUN ONCE
    # or run from the top if needed

large_df.insert(len(large_df.columns), 'status', binary_activity)

Nomralize RSSI [-1,0]

In [74]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1,0))
large_df['rssi'] = scaler.fit_transform(large_df[['rssi']])


In [79]:
# this function normalizes columns using z-score

def z_standardize(dataframe, *columns):
    for column in columns:
        col = str(column)
        dataframe[col] = (dataframe[col] - dataframe[col].mean()) / dataframe[col].std()
    return dataframe

# normalized most columns
# ".sample(frac=1)" part of the function just shuffles the data points around
    # can delete that part if needed
z_standardize(large_df, 'time', 'frontal accel', 'vertical accel', 'lateral accel', 'phase', 'frequency', 'consecutiveness', 'acceleration', 'freq').sample(frac=1)



,time,room,frontal accel,vertical accel,lateral accel,antenna id,rssi,phase,frequency,gender,consecutiveness,activity,acceleration,freq,status
18181,-0.547296,1,-0.948012,1.527441,0.386985,1,-0.582090,1.194140,-1.441573,0,-0.822632,1,-0.567449,-1.249305,0
30968,0.386479,1,0.501053,-0.911475,0.386985,4,-0.701493,-1.198657,-1.441573,1,-0.797232,3,-1.832855,0.666497,0
41084,0.268617,1,1.370512,-1.021089,0.465048,1,-0.731343,0.854630,0.047326,1,0.099384,3,1.840302,0.666497,0
7373,-0.139534,1,0.761987,-0.336004,0.178808,1,-0.522388,1.376200,-1.441573,0,0.945201,3,-0.446581,0.666497,0
51649,-0.447880,1,1.515352,-1.021089,0.386985,3,-0.537313,-0.921696,1.834004,1,-0.494973,3,2.457825,0.666497,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57371,0.257937,2,-0.716170,-0.034554,-1.694730,2,-0.268657,0.217766,-1.441573,0,0.292424,3,0.060519,0.666497,0
62750,1.832676,2,-1.034965,0.047634,-1.798696,2,-0.417910,-0.597674,-0.250454,0,-0.418774,3,0.157947,0.666497,0
4777,-0.906321,1,-1.440714,1.582185,0.204845,4,-0.537313,1.251787,-1.441573,0,-0.881052,1,-0.730638,-1.249305,0
45183,-0.858555,1,1.196507,-0.445611,-0.107419,3,-0.597015,0.697865,-1.441573,1,-0.875972,3,1.484812,0.666497,0


In [76]:
from sklearn.model_selection import train_test_split


X = large_df.drop(columns=['status'])
y = large_df['status']
train_data, test_data = train_test_split(X, test_size=0.25, random_state=42)
train_label, test_label = train_test_split(y, test_size=0.25, random_state=42)



### Building + Testing Model

**Hyper param testing**  
Martin  


In [77]:
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasRegressor

# TODO: add hyperparameters that we will be testing as parameters for this function (Martin)
def create_model(): 
    print("NOT IMPLEMENTED")
    # TODO: implement our nueral network structure using SGD as the optimization technique (Martin)

    # maybe use tf.keras.sequential here?
    model = 0
    return model
    
# TODO: add hyper parameters that we will be testing to param_grid (Martin)
param_grid = {
    "activation"
}

keras_model = KerasRegressor(build_fn=create_model, verbose=0)

# TODO: for GridSearchCV modify any other params that we need (scoring, cv, num_jobs, etc) (Martin)
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid)
grid_result = grid.fit(X_train_scaled, y_train)

# TODO: implement model evaluation and print/graph anything needed for visualization

ModuleNotFoundError: No module named 'keras.wrappers'